In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：用于在线预测的AutoML文本情感分析模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_text_sentiment_analysis_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_text_sentiment_analysis_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库创建文本情感分析模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)进行在线预测。

数据集

本教程使用的数据集是来自[data.world数据集](https://data.world) 的[Crowdflower Claritin-Twitter数据集](https://data.world/crowdflower/claritin-twitter)。您将在本教程中使用的数据集版本存储在公共云存储桶中。

### 目标

在本教程中，您将使用Vertex客户端库从Python脚本创建一个AutoML文本情感分析模型，并进行在线预测部署。您也可以使用`gcloud`命令行工具或在Google Cloud控制台上进行创建和部署模型。

所执行的步骤包括：

- 创建一个Vertex`数据集`资源。
- 训练模型。
- 查看模型评估。
- 将`模型`资源部署到`终端节点`资源。
- 进行预测。
- 取消部署`模型`。

成本

本教程使用Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
价格](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预计使用情况生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您已安装了Vertex客户端库和Google *cloud-storage*，您需要重新启动笔记本内核以便找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU运行时

*如果有这个选择的话，请确保你在GPU运行时中运行这个笔记本。在Colab中，选择* **Runtime > Change Runtime Type > GPU**

### 设置你的谷歌云项目

**无论你的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当你首次创建一个账户时，你会获得$300免费的计算/存储成本的信用。

2. [确保为你的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用Vertex APIs和Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [谷歌云SDK](https://cloud.google.com/sdk)已经安装在谷歌云笔记本中。

5. 在下方的单元格中输入你的项目ID。然后运行该单元格，确保Cloud SDK在这个笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter将以`!`为前缀的行视为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于本笔记本其余部分的操作。以下是Vertex支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不会使用多区域存储桶进行Vertex的训练。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持，请参见[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行直播教程会话，您可能正在使用一个共享测试账户或项目。为了避免用户在创建的资源中发生命名冲突，您可以为每个实例会话创建一个时间戳，并附加在将在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Google Cloud Notebook**，则您的环境已经经过身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证以通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

点击**创建服务帐号**。

在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务帐号访问项目**部分，点击角色下拉列表。在过滤框中输入"Vertex"，然后选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，然后选择**存储对象管理员**。

点击创建。包含您的密钥的JSON文件将下载到您的本地环境中。

在下面的单元格中将您的服务帐号密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和端点服务的顶点API服务终端点。
- `PARENT`：数据集、模型、作业、流水线和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML常量

设定专属于AutoML数据集和训练的常量：

- 数据集模式：告诉`数据集`资源服务数据集的类型是什么。
- 数据标记（注释）模式：告诉`数据集`资源服务数据是如何标记的。
- 数据集训练模式：告诉`管道`资源服务要为哪个任务（例如分类）训练模型。

In [ ]:
# Text Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/text_1.0.0.yaml"
# Text Labeling type
LABEL_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/ioformat/text_sentiment_io_format_1.0.0.yaml"
# Text Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_text_sentiment_1.0.0.yaml"

教程

现在您已经准备好开始创建自己的AutoML文本情感分析模型。

## 设置客户端

Vertex客户端库以客户端/服务器模型工作。在您的一边（Python脚本），您将创建一个客户端，向Vertex服务器发送请求并接收响应。

在本教程中，您将为工作流中的不同步骤使用不同的客户端。因此，请提前设置它们。

- 用于“数据集（Dataset）”资源的数据集服务。
- 用于“模型（Model）”资源的模型服务。
- 用于训练的管道服务。
- 用于部署的端点服务。
- 用于提供服务的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 数据集

现在您的客户准备就绪，在训练模型的第一步是创建一个托管的数据集实例，然后将您的标记数据上传到其中。

### 创建`Dataset`资源实例

使用辅助函数`create_dataset`来创建`Dataset`资源的实例。此函数执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个Vertex `Dataset`资源（`aip.Dataset`），包括以下参数：
   - `display_name`：您选择的可读名称。
   - `metadata_schema_uri`：数据集类型的模式。
3. 调用客户端数据集服务方法`create_dataset`，包括以下参数：
   - `parent`：您的`Database`、`Model`和`Endpoint`资源的Vertex位置根路径。
   - `dataset`：您创建的Vertex数据集对象实例。
4. 该方法返回一个`operation`对象。

`operation`对象是Vertex处理长时间运行操作的异步调用方式。虽然这一步通常很快，但在项目中首次使用时，可能会由于供应而有较长延迟。

您可以使用`operation`对象来获取操作（例如创建`Dataset`资源）的状态，或者取消操作，通过调用操作方法：

| 方法        | 描述                   |
| ----------- | -------------------- |
| result()    | 等待操作完成并以JSON格式返回结果对象。 |
| running()   | 返回操作是否仍在运行的True/False值。       |
| done()      | 返回操作是否已完成的True/False值。    |
| canceled()  | 返回操作是否已取消的True/False值。    |
| cancel()    | 取消操作（可能需要最多30秒）。   |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        dataset = aip.Dataset(
            display_name=name, metadata_schema_uri=schema, labels=labels
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("claritin-" + TIMESTAMP, DATA_SCHEMA)

现在保存您创建的“Dataset”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### 数据准备

对于文本，Vertex `Dataset` 资源有一些对您的文本数据的要求。

- 文本示例必须存储在 CSV 或 JSONL 文件中。

#### CSV

对于文本情感分析，CSV文件有一些要求：

- 无标题。
- 第一列是文本示例或云存储路径到文本文件。
- 第二列是标签（即情感）。
- 第三列是最大情感值。例如，如果范围是0到3，则最大值为3。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为在云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/language/claritin.csv"
SENTIMENT_MAX = 4

快速查看您的数据

您将使用存储在公共云存储桶中的Crowdflower Claritin-Twitter数据集的一个版本，使用CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 导入数据

现在，将数据导入到您的Vertex Dataset资源中。使用这个助手函数`import_data`来导入数据。该函数执行以下操作：

- 使用`Dataset`客户端。
- 调用客户端方法`import_data`，并传递以下参数：
 - `name`：您为`Dataset`资源指定的可读名称（例如，claritin）。
 - `import_configs`：导入配置。

- `import_configs`：包含字典的Python列表，具有键/值条目如下：
 - `gcs_sources`：一个包含一个或多个索引文件路径的URI列表。
 - `import_schema_uri`：标识标签类型的架构。

`import_data()`方法返回一个长时间运行的`operation`对象。这将需要几分钟才能完成。如果您在一个现场教程中，现在是一个问问题或休息的好时机。

In [ ]:
def import_data(dataset, gcs_sources, schema):
    config = [{"gcs_source": {"uris": gcs_sources}, "import_schema_uri": schema}]
    print("dataset:", dataset_id)
    start_time = time.time()
    try:
        operation = clients["dataset"].import_data(
            name=dataset_id, import_configs=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation
    except Exception as e:
        print("exception:", e)
        return None


import_data(dataset_id, [IMPORT_FILE], LABEL_SCHEMA)

## 训练模型

现在，使用您的 Vertex `Dataset` 资源训练一个 AutoML 文本情感分析模型。要训练模型，请执行以下步骤：

1. 为 `Dataset` 资源创建一个 Vertex 训练管道。
2. 执行管道以开始训练。

### 创建一个训练流水线

您可能会问，我们用流水线做什么？通常在工作（如训练）有多个步骤时，我们会按照顺序使用流水线：执行步骤A，执行步骤B等等。通过将这些步骤放入流水线中，我们可以获得以下好处：

1.可重用于后续训练工作。
2.可以容器化并作为批处理作业运行。
3.可以进行分布式处理。
4.所有步骤都与相同的流水线作业相关联，用于跟踪进展。

使用这个辅助函数 `create_pipeline`，该函数接受以下参数：

- `pipeline_name`：用于流水线作业的人类可读名称。
- `model_name`：模型的人类可读名称。
- `dataset`：Vertex 完全限定的数据集标识符。
- `schema`：数据集标签（注释）训练模式。
- `task`：描述训练作业要求的字典。

该辅助函数调用 `Pipeline` 客户端服务的 `create_pipeline` 方法，该方法接受以下参数：

- `parent`：`Dataset`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
- `training_pipeline`：流水线训练作业的完整规范。

现在让我们更深入地了解构建 `training_pipeline` 规范的*最低*要求：

- `display_name`：用于流水线作业的人类可读名称。
- `training_task_definition`：数据集标签（注释）训练模式。
- `training_task_inputs`：描述训练作业要求的字典。
- `model_to_upload`：模型的人类可读名称。
- `input_data_config`：数据集规范。
 - `dataset_id`：仅限 Vertex 数据集标识符（非完全限定），这是完全限定标识符的最后部分。
 - `fraction_split`：如果指定，数据集用于训练、测试和验证的百分比。否则，百分比将由 AutoML 自动选择。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务要求

接下来，构建任务要求。与需要传入 Python（类似 JSON 格式）字典的其他参数不同，`task` 字段需要传入 Google protobuf Struct，它与 Python 字典非常相似。使用 `json_format.ParseDict` 方法进行转换。

我们需要指定的最小字段是：

- `sentiment_max`：情感的最大值（例如，4）。

最后，通过调用帮助函数 `create_pipeline` 来创建流水线，该函数将返回一个训练流水线对象的实例。

In [ ]:
PIPE_NAME = "claritin_pipe-" + TIMESTAMP
MODEL_NAME = "claritin_model-" + TIMESTAMP

task = json_format.ParseDict(
    {
        "sentiment_max": SENTIMENT_MAX,
    },
    Value(),
)

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的培训管道的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取有关培训管道的信息

现在只获取此培训管道实例的管道信息。通过调用作业客户端服务的`get_training_pipeline`方法，使用以下参数，辅助函数会获取该作业的信息：

- `name`：Vertex 完全合格的管道标识符。

当模型训练完成时，管道状态将为`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

部署

培训上述模型可能需要多达180分钟的时间。

一旦您的模型培训完成，您可以通过将`end_time`减去`start_time`来计算训练模型所花费的实际时间。对于您的模型，您需要知道完全限定的Vertex模型资源标识符，即管道服务为其分配的。您可以从返回的管道实例中获取此信息，字段为`model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

模型信息

现在您的模型已经训练完成，您可以获取一些关于您的模型的信息。

评估模型资源

现在找出模型服务认为你的模型有多好。作为训练的一部分，数据集的一部分被设置为测试（留出）数据，管线服务使用该数据来评估模型。

### 列出所有切片的评估

请使用此辅助函数 `list_model_evaluations`，该函数接受以下参数：

- `name`：`Model` 资源的 Vertex 完全限定模型标识符。

此辅助函数使用模型客户端服务的 `list_model_evaluations` 方法，该方法接受相同的参数。调用的响应对象是一个列表，其中每个元素都是一个评估指标。

对于每个评估 -- 您可能只有一个，然后我们打印出每个指标的所有键名称，并且对于一个小的集合（`meanAbsoluteError` 和 `precision`），您将打印出结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("meanAbsoluteError", metrics["meanAbsoluteError"])
        print("precision", metrics["precision"])

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

## 部署`Model`资源

现在部署您使用AutoML创建的已训练的Vertex`Model`资源。这需要两个步骤：

1. 为部署`Model`资源创建一个`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源。

### 创建一个 `Endpoint` 资源

使用这个辅助函数 `create_endpoint` 来创建一个端点，用于部署模型以进行预测服务，需要以下参数：

- `display_name`: 为 `Endpoint` 资源提供一个易读的名称。

这个辅助函数使用了端点客户端服务的 `create_endpoint` 方法，该方法接受以下参数：

- `display_name`: 为 `Endpoint` 资源提供一个易读的名称。

创建一个 `Endpoint` 资源会返回一个长时间运行的操作，因为需要一些时间来为提供服务的 `Endpoint` 资源进行设置。您可以调用 `response.result()`，这是一个同步调用，当 Endpoint 资源准备好时会返回结果。该辅助函数返回了 `Endpoint` 资源的 Vertex 完全限定标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "claritin_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的 “Endpoint” 资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

处理您的在线预测请求时，您有几种计算实例扩展的选择：

- 单个实例：在线预测请求在单个计算实例上进行处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：在线预测请求分配到您手动指定的固定数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同数量的节点。当模型首次部署到实例上时，固定数量的计算实例将被预配，并且在线预测请求将均匀分布在它们之间。

- 自动扩展：在线预测请求分配到可扩展数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）数量设置为模型首次部署时需预配和取消预配的实例数，并根据负载情况将计算实例的最大（`MAX_NODES`）数量设置为需预配的实例数。

计算实例的最小数量对应于字段`min_replica_count`，最大数量对应于字段`max_replica_count`，在您随后的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 将`Model`资源部署到`Endpoint`资源

使用这个帮助函数`deploy_model`来将您为提供预测创建的`Model`资源部署到`Endpoint`资源，具有以下参数：

- `model`：要从训练管道上传（部署）的模型的 Vertex 完全合格的模型标识符。
- `deploy_model_display_name`：部署模型的人类可读名称。
- `endpoint`：要将模型部署到的 Vertex 完全合格的端点标识符。

该帮助函数调用`Endpoint`客户端服务的方法`deploy_model`，它接受以下参数：

- `endpoint`：要将`Model`资源部署到的 Vertex 完全合格的`Endpoint`资源标识符。
- `deployed_model`：部署模型的要求规范。
- `traffic_split`：在端点上流量的百分比，指定为一个或多个键/值对的字典。
   - 如果只有一个模型，则指定为**{"0": 100}**，其中"0"表示正在上传的模型，100表示100%的流量。
   - 如果端点上已有现有模型，流量将被分拆给它们，使用`model_id`来指定为**{"0": 百分比，model_id: 百分比，...}**，其中`model_id`是要部署到的端点的现有模型的模型标识。百分比必须总和为100。

现在让我们更深入地了解`deployed_model`参数。该参数被指定为一个Python字典，具有最小所需字段：

- `model`：要部署的（上传）模型的 Vertex 完全合格的模型标识符。
- `display_name`：部署模型的人类可读名称。
- `disable_container_logging`：这会禁用容器事件的记录，例如执行失败（默认为启用容器日志记录）。通常在调试部署时启用容器日志记录，然后在部署到生产环境时禁用。
- `automatic_resources`：这指的是多少个冗余计算实例（副本）。在这个示例中，我们将其设置为一个（不复制）。

#### 流量分流

现在让我们更深入地了解`traffic_split`参数。该参数被指定为一个Python字典。一开始可能有点令人困惑。让我解释一下，您可以部署您的模型的多个实例到一个端点，然后设置每个实例所占的比例（百分比）。

为什么要这样做？也许您已经在生产中部署了一个先前版本 -- 让我们称之为 v1。您在 v2 上获得了更好的模型评估，但在部署到生产环境之前并不确定它真的更好。因此，在流量分流的情况下，您可能希望将 v2 部署到与 v1 相同的端点，但它只占到比如10%的流量。这样，您可以在不干扰大多数用户的情况下监控其表现，直到做出最终决定。

#### 响应

该方法返回一个长期运行的操作`response`。我们将通过调用`response.result()`同步等待操作完成，这将阻塞直到模型部署完成。如果这是第一次将模型部署到端点，可能需要额外几分钟来完成资源的规划。

In [ ]:
DEPLOYED_NAME = "claritin_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "automatic_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

## 进行在线预测请求

现在对您部署的模型进行在线预测。

获取测试项目

您将从数据集中选择一个任意的示例作为测试项目。不必担心该示例很可能在训练模型时被使用过 — 我们只是想演示如何进行预测。

In [ ]:
test_item = ! gsutil cat $IMPORT_FILE | head -n1
if len(test_item[0]) == 3:
    _, test_item, test_label, max = str(test_item[0]).split(",")
else:
    test_item, test_label, max = str(test_item[0]).split(",")

print(test_item, test_label)

### 做出预测

现在您有一个测试项目。使用这个辅助函数 `predict_item`，它接受以下参数：

- `filename`：测试项目的云存储路径。
- `endpoint`：`Model`资源部署的`Endpoint`资源的 Vertex 完全限定标识符。
- `parameters_dict`：用于提供预测结果的附加过滤参数。

此函数调用预测客户端服务的 `predict` 方法，其中包括以下参数：

- `endpoint`：`Model`资源部署的`Endpoint`资源的 Vertex 完全限定标识符。
- `instances`：要预测的实例（文本文件）的列表。
- `parameters`：用于提供预测结果的附加过滤参数。*注意*，文本模型不支持额外的参数。

#### 请求

每个实例的格式如下：

    { 'content': text_item }

由于 `predict()` 方法可以接受多个项目（实例），您将您的单个测试项目发送为一个测试项目列表。最后一步，将实例列表打包成谷歌的 protobuf 格式 -- 这是您传递给 `predict()` 方法的内容。

#### 响应

`response` 对象返回一个列表，其中列表中的每个元素对应请求中的相应文本。您将在每次预测的输出中看到 -- 在我们的例子中只有一个：

-情感评分

In [ ]:
def predict_item(data, endpoint, parameters_dict):

    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{"content": data}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", dict(prediction))
    return response


response = predict_item(test_item, endpoint_id, None)

## 取消部署 `Model` 资源

现在从提供服务的 `Endpoint` 资源中取消部署您的 `Model` 资源。使用这个辅助函数 `undeploy_model`，它接受以下参数：

- `deployed_model_id`：当 `Model` 资源被部署到时，终端服务返回的模型部署标识符。
- `endpoint`：`Model` 部署到的 `Endpoint` 资源的 Vertex 全限定标识符。

该函数调用终端客户服务的 `undeploy_model` 方法，带有以下参数：

- `deployed_model_id`：当 `Model` 资源被部署时，终端服务返回的模型部署标识符。
- `endpoint`：`Model` 资源被部署到的 `Endpoint` 资源的 Vertex 全限定标识符。
- `traffic_split`：如何在 `Endpoint` 资源上分配流量给其余部署的模型。

由于这是 `Endpoint` 资源上唯一部署的模型，您可以通过将其设置为 {} 而将 `traffic_split` 留空。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理

要清理此项目中使用的所有GCP资源，您可以删除您用于教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批量作业
- 定制作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME